In [3]:
import json
import os
import matplotlib.pyplot as plt
import numpy as np
import heapq
from tqdm import tqdm

def read_jsonl(path):
    examples = []
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if line:
                try:
                    ex = json.loads(line)
                    examples.append(ex)
                except Exception as e:
                    print(e)
    return examples

def get_image_from_problem(problem):
    img_folder='TRAINING'
    img_list=[]
    frame_list=[]
    reason_list=[]
    ids=[]
    for i in read_jsonl('predictions_whole/articulation-annotations.jsonl'):
        for j in i['articulations']:
            if problem in j['reasoning']:
                img_list.append(i['id'])
                frame_list.append(j['text'])
                reason_list.append(j['reasoning'])
                ids.append(i['id'])
    print(len(img_list))
    for x, f_name in enumerate(img_list):
      try:
        image_path = os.path.join(img_folder, f_name)
      except:
        print('Not found')
      image = plt.imread(image_path)
      print(frame_list[x])
      print(reason_list[x])
      plt.imshow(image)
      plt.title(frame_list[x])
      plt.xlabel(ids[x])
      #plt.axis('off')
      plt.show()

def get_frame_from_problem(problem):
    img_folder='TRAINING'
    img_list=[]
    frame_list=[]
    reason_list=[]
    ids=[]
    for i in read_jsonl('predictions_whole/articulation-annotations.jsonl'):
        for j in i['articulations']:
            if problem in j['reasoning']:
                img_list.append(i['id'])
                frame_list.append(j['text'])
                reason_list.append(j['reasoning'])
                ids.append(i['id'])
    return frame_list, reason_list

In [4]:
import time
from typing import Optional
import json
import base64
from collections import defaultdict


def read_jsonl(path):
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if line:
                ex = json.loads(line)
                yield ex

def write_jsonl(path, data):
    with open(path, "w") as f:
        for i, ex in enumerate(data):
            try:
                f.write(json.dumps(ex) + "\n")
            except TypeError as e:
                print(f"Error writing element at index {i}: {ex}")
                print(f"TypeError: {e}")

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def write_to_jsonl(data, filename):
    with open(filename, 'w') as file:
        for key, value in data.items():
            for entry in value:
                record = {"problem": key}
                record.update(entry)
                file.write(json.dumps(record) + "\n")

def read_from_jsonl_ind(filename):
    data = defaultdict(list)

    with open(filename, 'r') as file:
        for line in file:
            record = json.loads(line.strip())
            problem = record.pop("problem")
            data[problem].append(record)

    # Convert defaultdict back to a regular dict
    return dict(data)

start of relations

In [5]:
updated_merged_dict=read_from_jsonl_ind('updated_merged_dict_final.jsonl')
subproblems=list(read_jsonl('subproblems_1c.jsonl'))

subproblems_list=[]
for subproblem in subproblems:

  if 'subproblem(' not in subproblem['response']:
    continue
  pt=(subproblem['response'].split('subproblem(')[1]).split(')')[0].split(', ')
  if len(pt)>1:
    subproblems_list.append(tuple(pt))


In [ ]:
import random

def write_frame_examples(updated_dict, num_examples_per_problem, output_file):
    with open(output_file, 'w') as file:
        for problem, frame_list in updated_dict.items():
            # Print problem statement
            file.write(f"Problem: {problem}\n")

            # Select a random sample of frames, or use all frames if fewer than the requested number
            selected_frames = random.sample(frame_list, min(len(frame_list), num_examples_per_problem))

            for frame in selected_frames:
                file.write(f"  Frame ID: {frame['frame_id']}\n")
                file.write(f"  Frame: {frame['frame']}\n")
                file.write(f"  Reasoning: {frame['reasoning']}\n\n")

            file.write("\n")  # Add extra line for readability
def find_problem_by_frame(problem_dict, target_frame):
    for problem, details in problem_dict.items():
      for x in details:
        if x["frame"] == target_frame:
          return problem, x

    return None
write_frame_examples(updated_merged_dict,5,'problem_frame_ex.txt')

In [ ]:
from collections import defaultdict
problems=list(updated_merged_dict.keys())
hierarchical_graph = defaultdict(list)

for child, parent in subproblems_list:
    hierarchical_graph[parent].append(child)

# Dictionaries to store the relationships
parent_to_children = defaultdict(list)
child_to_parent = {}

# Build the graph relationships
for child, parent in subproblems_list:
    parent_to_children[parent].append(child)
    child_to_parent[child] = parent

# Optional: Add isolated problems without parents (nodes with no incoming edges)
for problem in problems:
    if problem not in hierarchical_graph and problem not in [child for child, parent in subproblems_list]:
        hierarchical_graph[problem] = []

# Function to recursively print the hierarchy
def print_hierarchy(graph, parent, level=0):
    print("  " * level + parent)
    for child in graph[parent]:
        print_hierarchy(graph, child, level + 1)

# Find root nodes (problems that have no parents)
roots = [problem for problem in hierarchical_graph if problem not in [child for child, parent in subproblems_list]]

# Print the hierarchical graph
for root in roots:
    print_hierarchy(hierarchical_graph, root)


patriarchal attitudes
  enforced gender norms
    double standards
    gender essentialism
  promoting infidelity
  undermining women's capabilities
  wage disparity
  male validation
  intellectual degradation
  incest
  conditional respect
  toxic masculinity
  dismissing women's rights
    undermining women's rights movements
  women subjugation
  demeaning aspirations
  patriarchal control
    possessiveness
    gatekeeping
    coercion
    ownership
    policing women's bodies
    forced marriage
objectification
  sexualization
    sexual innuendo
    sexual entitlement
    age-related sexualization
  unrealistic beauty standards
discrimination of women
  ableism
  transphobia
  neo sexism
  appearance-based discrimination
  classism
  exclusion
  workplace discrimination
  racism
  homophobia
  ageism
  reverse sexism
  constrained gender identity
  intersectional prejudice
disrespect towards women
  dismissiveness
    invalidating women's experiences
  cultural insensitivity
  d

In [ ]:
def find_problem_by_frame(problem_dict, target_frame):
    for problem, details in problem_dict.items():
      for x in details:
        if x["frame"] == target_frame:
          return problem, x

    return None

def print_hierarchy_frame_count(child_to_parent, parent_to_child, updated_merged_dict):
    # Step 1: Find roots (nodes with no parents)
    all_nodes = set(parent_to_child.keys()).union(set(child_to_parent.keys()))
    roots = [node for node in all_nodes if node not in child_to_parent]

    # Step 2: Depth-First Search (DFS) to traverse hierarchy in order
    def dfs(node, level=0):
        # Get the number of frames from updated_merged_dict
        num_frames = len(updated_merged_dict.get(node, []))
        print(f"{' ' * level * 2}Key: {node}, Number of frames: {num_frames}")

        # Recursively visit each child in the hierarchy
        for child in parent_to_child.get(node, []):
            dfs(child, level + 1)

    # Step 3: Traverse from each root in hierarchical order
    for root in roots:
        dfs(root)
# Example usage
print_hierarchy_frame_count(child_to_parent, parent_to_children,updated_merged_dict)

In [ ]:
def get_image_from_problem(problem):
    img_folder='TRAINING'
    img_list=[]
    frame_list=[]
    reason_list=[]
    ids=[]
    for i in read_jsonl('predictions_whole/articulation-annotations.jsonl'):
        for j in i['articulations']:
            if problem in j['reasoning']:
                img_list.append(i['id'])
                frame_list.append(j['text'])
                reason_list.append(j['reasoning'])
                ids.append(i['id'])
    print(len(img_list))
    for x, f_name in enumerate(img_list):
      try:
        image_path = os.path.join(img_folder, f_name)
      except:
        print('Not found')
      image = plt.imread(image_path)
      print(frame_list[x])
      print(reason_list[x])
      plt.imshow(image)
      plt.title(frame_list[x])
      plt.xlabel(ids[x])
      #plt.axis('off')
      plt.show()

In [ ]:
def get_frames(img_id):
  for i in read_jsonl('predictions_whole/articulation-annotations.jsonl'):
    if i['id']==img_id:
      return i['articulations']
def get_frames2(frame):
  for i in read_jsonl('predictions_whole/articulation-annotations.jsonl'):
    for j in i['articulations']:
      if j['text']==frame:
        return i['id']

Evaluation

In [6]:
from itertools import combinations

def get_parent_pairs(subproblem_parent_pairs):
    """Returns a dictionary where keys are subproblems and values are their parents."""
    parent_dict = {}
    for subproblem, parent in subproblem_parent_pairs:
        parent_dict[subproblem] = parent
    return parent_dict

def get_subproblem_pairs(parent_dict):
    """Generate all possible pairs of subproblems and check if they have the same parent."""
    pairs = set()
    subproblems = list(parent_dict.keys())

    for sp1, sp2 in combinations(subproblems, 2):
        if parent_dict[sp1] == parent_dict[sp2]:
            pairs.add((sp1, sp2))

    return pairs

def calculate_rand_index_for_subproblems(pred_pairs, gold_pairs, all_subproblems):
    # Step 1: Generate all possible pairs of subproblems
    all_pairs = set(combinations(sorted(all_subproblems), 2))

    # Step 2: Calculate counts for n11, n00, n10, n01
    n11 = len(pred_pairs & gold_pairs)  # In the same cluster in both
    n00 = len(all_pairs - (pred_pairs | gold_pairs))  # In different clusters in both
    n10 = len(pred_pairs - gold_pairs)  # In the same cluster in predicted, different in gold
    n01 = len(gold_pairs - pred_pairs)  # In the same cluster in gold, different in predicted

    # Step 3: Calculate Rand Index
    rand_index = (n11 + n00) / (n11 + n10 + n01 + n00) if (n11 + n10 + n01 + n00) > 0 else 0
    return rand_index





In [7]:
pred_subproblem=list(read_jsonl('subproblems_1cb.jsonl'))
gold_subproblem=list(read_jsonl('subproblems_1c.jsonl'))

gold_subproblems_list=[]
for subproblem in gold_subproblem:

  if 'subproblem(' not in subproblem['response']:
    continue
  pt=(subproblem['response'].split('subproblem(')[1]).split(')')[0].split(', ')
  if len(pt)>1:
    gold_subproblems_list.append(tuple(pt))

pred_subproblems_list=[]
for subproblem in pred_subproblem:

  if 'subproblem(' not in subproblem['response']:
    continue
  pt=(subproblem['response'].split('subproblem(')[1]).split(')')[0].split(', ')
  if len(pt)>1:
    pred_subproblems_list.append(tuple(pt))

pred_parent_dict = get_parent_pairs(pred_subproblems_list)
gold_parent_dict = get_parent_pairs(gold_subproblems_list)

# Step 2: Generate subproblem pairs (same parent) for both predicted and gold
pred_pairs = get_subproblem_pairs(pred_parent_dict)
gold_pairs = get_subproblem_pairs(gold_parent_dict)

# Step 3: Get all subproblems from both taxonomies
all_subproblems = set(pred_parent_dict.keys()).union(gold_parent_dict.keys())

# Step 4: Calculate Rand Index
rand_index = calculate_rand_index_for_subproblems(pred_pairs, gold_pairs, all_subproblems)
print(f"Rand Index for (subproblem, parent) pairs: {rand_index:.4f}")

Rand Index for (subproblem, parent) pairs: 0.9625


In [ ]:
from collections import defaultdict

def compute_ancestral_pairs(pairs):
    ancestors = defaultdict(set)

    # Add direct parent relationships
    for child, parent in pairs:
        ancestors[child].add(parent)

    # Add indirect ancestors (transitive closure)
    for child in ancestors:
        to_process = list(ancestors[child])
        while to_process:
            ancestor = to_process.pop()
            if ancestor in ancestors:
                for grandparent in ancestors[ancestor]:
                    if grandparent not in ancestors[child]:
                        ancestors[child].add(grandparent)
                        to_process.append(grandparent)
    return ancestors

def compute_ancestor_f1(gold_pairs, predicted_pairs):
    gold_ancestors = compute_ancestral_pairs(gold_pairs)
    predicted_ancestors = compute_ancestral_pairs(predicted_pairs)

    # Calculate true positives
    tp = 0
    total_predicted = 0
    total_gold = 0

    # Count the number of correct (is-a) pairs
    for child, ancestors in predicted_ancestors.items():
        total_predicted += len(ancestors)
        if child in gold_ancestors:
            tp += len(ancestors & gold_ancestors[child])

    for child, ancestors in gold_ancestors.items():
        total_gold += len(ancestors)

    # Precision, Recall, F1
    precision = tp / total_predicted if total_predicted > 0 else 0
    recall = tp / total_gold if total_gold > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

# Example usage
gold_standard = [("subproblem1", "parent1"), ("subproblem2", "parent1"), ("subproblem3", "parent2")]
predicted_taxonomy = [("subproblem1", "parent1"), ("subproblem2", "parent2"), ("subproblem3", "parent1")]

Pa, Ra, F1a = compute_ancestor_f1(gold_standard, predicted_taxonomy)

print(f"Ancestor Precision: {Pa:.2f}, Ancestor Recall: {Ra:.2f}, Ancestor F1: {F1a:.2f}")


In [ ]:
def compute_edge_f1(gold_pairs, predicted_pairs):
    gold_set = set(gold_pairs)
    predicted_set = set(predicted_pairs)

    # True Positives (correct predicted edges)
    tp = len(gold_set & predicted_set)

    # Total predicted and total gold edges
    total_predicted = len(predicted_set)
    total_gold = len(gold_set)

    # Precision, Recall, F1
    precision = tp / total_predicted if total_predicted > 0 else 0
    recall = tp / total_gold if total_gold > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

# Example usage
gold_standard = [("subproblem1", "parent1"), ("subproblem2", "parent1"), ("subproblem3", "parent2")]
predicted_taxonomy = [("subproblem1", "parent1"), ("subproblem2", "parent2"), ("subproblem3", "parent1")]

Pe, Re, F1e = compute_edge_f1(gold_standard, predicted_taxonomy)

print(f"Edge Precision: {Pe:.2f}, Edge Recall: {Re:.2f}, Edge F1: {F1e:.2f}")
